In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d agrigorev/clothing-dataset-full

100% 6.49G/6.50G [02:14<00:00, 55.2MB/s]
100% 6.50G/6.50G [02:14<00:00, 51.8MB/s]


In [ ]:

from zipfile import ZipFile
file_name = 'clothing-dataset-full.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Import the dataset 
labels = pd.read_csv('/content/images.csv')

In [ ]:
labels.head()

,image,sender_id,label,kids
0,4285fab0-751a-4b74-8e9b-43af05deee22,124,Not sure,False
1,ea7b6656-3f84-4eb3-9099-23e623fc1018,148,T-Shirt,False
2,00627a3f-0477-401c-95eb-92642cbe078d,94,Not sure,False
3,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa,43,T-Shirt,False
4,3b86d877-2b9e-4c8b-a6a2-1d87513309d0,189,Shoes,False


In [ ]:
# Classes and their counts within the dataset
labels['label'].value_counts()

T-Shirt       1011
Longsleeve     699
Pants          692
Shoes          431
Shirt          378
Dress          357
Outwear        312
Shorts         308
Not sure       228
Hat            171
Skirt          155
Polo           120
Undershirt     118
Blazer         109
Hoodie         100
Body            69
Other           67
Top             43
Blouse          23
Skip            12
Name: label, dtype: int64

In [ ]:
labels.loc[labels['label']=='Not sure','label'] = 'Not_sure'

In [ ]:
labels['image'] = labels['image'] + '.jpg'

In [ ]:
labels['label_cat'] = labels['label'] + ' ' + labels['kids'].astype(str)

In [ ]:
label_df = labels[['image', 'label_cat']]

In [ ]:
label_df.head()

,image,label_cat
0,4285fab0-751a-4b74-8e9b-43af05deee22.jpg,Not_sure False
1,ea7b6656-3f84-4eb3-9099-23e623fc1018.jpg,T-Shirt False
2,00627a3f-0477-401c-95eb-92642cbe078d.jpg,Not_sure False
3,ea2ffd4d-9b25-4ca8-9dc2-bd27f1cc59fa.jpg,T-Shirt False
4,3b86d877-2b9e-4c8b-a6a2-1d87513309d0.jpg,Shoes False


In [ ]:
path = '/content'
def get_x(r): return path+'/images_original/'+r['image'] # create path to open images in the original folder
def get_y(r): return r['label_cat'].split(' ') # split the labels using space as a delimitter

In [ ]:
# Create DataBlock
dblock = DataBlock(blocks = (ImageBlock, MultiCategoryBlock),
                  get_x = get_x, get_y = get_y,
                  item_tfms = RandomResizedCrop(128, min_scale=0.35))  # ensure every item is of the same size
dls = dblock.dataloaders(label_df) # collates items from dataset into minibatches

In [ ]:
dls.show_batch(nrows=3, ncols=3)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.2))
learn.fine_tune(10, base_lr=3e-3)

In [ ]:
# Get Predictions and target variables
preds,targs = learn.get_preds()

In [ ]:
xs = torch.linspace(0.01,0.99,50)
accs = [accuracy_multi(preds, targs, thresh=i, sigmoid=False) for i in xs] # get_preds applies sigmoid activation function
plt.plot(xs,accs);

In [ ]:
learn.export(Path("/content/drive/MyDrive/DL Projects/export.pkl"))

In [ ]:
upload = widgets.FileUpload()
out_image = widgets.Output()
prediction = widgets.Label()
run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(upload.data[-1])
    out_image.clear_output()
    with out_image: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn.predict(img)
    pred0 = pred[0]
    pred1 = pred[1]
    if pred0=='False':
        prediction.value = f'This is a {pred1} for adults.'
    else:
        prediction.value = f'This is a {pred1} for kids.'

run.on_click(on_click_classify)


In [ ]:
# Use Virtual Box to encapsulate the iPython widgets
VBox([widgets.Label('Upload a picture of a pice of clothing!'), 
      upload, run, out_image, prediction])